 Author: Moshe Silverstein <br/>
 Date: 7-17 <br/>
 Data Downloaded: 5-30-2017 <br/>
 Data Source: https://bioplex.hms.harvard.edu/interactions.php

 Reviewer: Charles Dai <br>
 Updated: 6-20

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

 ### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

'3.8.0 (default, Oct 28 2019, 16:14:01) \n[GCC 8.3.0]'

 # Initialization

 ### Load Mapping Dictionaries

In [6]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:09<00:00,  3.09s/it]


 ### Output Path

In [7]:
output_name = 'bioplex'

path = 'Output/BioPlex'
if not os.path.exists(path):
    os.makedirs(path)

In [8]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the BioPlex PPI Data Set',
) %}

 # Load Data

In [23]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.tsv$',
    name='interactions', 
    label='BioPlex Interaction List', 
    default='Input/BioPlex/BioPlex_293T_Network_10K_Dec_2019.tsv',
    section='data')
}}, sep='\t', usecols=['SymbolA', 'SymbolB'], index_col=0)

```python

df = pd.read_csv('Input/BioPlex/BioPlex_293T_Network_10K_Dec_2019.tsv', sep='\t', usecols=['SymbolA', 'SymbolB'], index_col=0)
```

In [24]:
df.head()

,SymbolB
SymbolA,
ADA,POTEF
BEND7,RPL13
BEND7,RBBP4
BEND7,RPL36
BEND7,RPL4


In [25]:
df.shape

(118162, 1)

 # Filter Data

 ## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [26]:
df = uf.mapgenesymbols(df, symbol_lookup)
df = df.reset_index().set_index('SymbolB')
df = uf.mapgenesymbols(df, symbol_lookup)
df = df.reset_index().set_index('SymbolA')
df.shape

100%|██████████| 118033/118033 [00:00<00:00, 580532.92it/s]


(117738, 1)

 # Analyze Data

 ## Create Binary Matrix

In [ ]:
binary_matrix = uf.createBinaryMatrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.saveData(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

 ## Create Gene List

In [ ]:
gene_list = uf.createGeneList(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.saveData(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

 ## Create Attribute List

In [ ]:
attribute_list = uf.createAttributeList(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.saveData(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

 ## Create Gene and Attribute Set Libraries

In [ ]:
uf.createUpGeneSetLib(binary_matrix, path, output_name + '_gene_up_set')

In [ ]:
uf.createUpAttributeSetLib(binary_matrix, path, 
                           output_name + '_attribute_up_set')

 ## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.saveData(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

 ## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.saveData(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

 ## Create Gene-Attribute Edge List

In [ ]:
uf.createGeneAttributeEdgeList(binary_matrix, attribute_list, gene_list, 
                               path, output_name + '_gene_attribute_edge_list')

 # Create Downloadable Save File

In [ ]:
uf.createArchive(path)

 ### Link to download output files: [click here](./output_archive.zip)